# Load necessary packages

In [2]:
# import of packages
from proj03 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from sklearn.utils import resample


# autoreload for easier debugging
%load_ext autoreload
%autoreload 2

# Import raw data

In [3]:
# import data
baseline = pd.read_stata('data/baseline.dta')
bok_inflation = pd.read_stata('data/BOK_inflation.dta')
cleanpricedata_y1y2 = pd.read_stata('data/cleanPriceData_Y1Y2.dta')
intensity_obs_short = pd.read_stata('data/intensity_obs_short.dta')
lrfu_select_dataset = pd.read_stata('data/LRFU_select_dataset.dta')
ms1ms2_pooled = pd.read_stata('data/MS1MS2_pooled.dta')
repayment_datay1 = pd.read_stata('data/repayment_dataY1.dta')

# Creating the Tables

## Create Table 1

We start by cleaning the ms1ms2_pooled and baseline data.

In [4]:
# clean ms1ms2_pooled (drop if MS !=2, keep columns oafid and treatMS1MS2, group by oafid and take mean and rename) 
ms1ms2_pooled_clean = ms1ms2_pooled[ms1ms2_pooled['MS']==2]
ms1ms2_pooled_clean = ms1ms2_pooled_clean[['oafid', 'treatMS1MS2']]
print(ms1ms2_pooled_clean.shape[0])
ms1ms2_pooled_clean = ms1ms2_pooled_clean.groupby('oafid', as_index=False).mean()
ms1ms2_pooled_clean.rename(columns={'treatMS1MS2': 'treat13'}, inplace=True)
print(ms1ms2_pooled_clean.shape[0])


2993
1019


For the baseline data we note that some of the columns have already been renamed with the suffix `_base` however and thus need to account for this. We however, assume that the data have not been altered in any other way compared to what the do in the `do` file.

In [5]:
# clean baseline data (the stata code indicates that the variables columns 'businessprofitmonth' and 'delta' should be kept, however they have already been renamed to 'businessprofitmonth_base' and 'delta_base')
base_cols = ['oafid', 'logtotcons_base', 'male', 'num_adults', 'num_schoolchildren', 'finished_primary',
                   'finished_secondary', 'cropland', 'num_rooms', 'schoolfees', 'totcons_base', 'logpercapcons_base',
                   'total_cash_savings_base', 'total_cash_savings_trimmed', 'has_savings_acct', 'taken_bank_loan',
                   'taken_informal_loan', 'liquidWealth', 'wagepay', 'businessprofitmonth_base', 'price_avg_diff_pct',
                   'price_expect_diff_pct', 'harvest2011', 'netrevenue2011', 'netseller2011', 'autarkic2011',
                   'maizelostpct2011', 'harvest2012', 'correct_interest', 'digit_recall', 'maizegiver', 'delta_base', 'treatment']
baseline_clean = baseline[base_cols].copy()

""" WE SURE ABOUT THIS? """
# we assume that the variable 'delta_base' does not already have the correct value thus the following is not commented out
baseline['delta_base'] = 1 - baseline['delta_base']

# rename columns
baseline_clean.columns = [col + '_base' if not col.endswith('_base') and col != 'oafid' and col != 'treatment' else col for col in baseline_clean.columns]
baseline_clean.rename(columns={'treatment': 'treatment2012'}, inplace=True)

# generate treat12 as bool for treatment and control in 2012
baseline_clean['treat12'] = baseline_clean['treatment2012'].apply(lambda x: x in ['T1', 'T2'])
baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan

base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_clean, on='oafid', how='left')


/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_96399/232695012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan


Now we can merge the two datasets.

In [6]:
# merge baseline_clean and ms1ms2_pooled_clean on oafid
# base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_clean, on='oafid', how='left')

#  # The rest of the code is not what is done in Stata but it I believe it is not used for table 1 (and I believe it is not used for only 'in_sample_Y2' is only used for Table F.1 – the join should just be a left join)
# base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_clean, on='oafid', how='outer', indicator=True)

# # Drop rows that are only in the using dataset (equivalent to 'merge_base == 2' in Stata)
# base_ms1ms2_pool = base_ms1ms2_pool[base_ms1ms2_pool['_merge'] != 'right_only']

# # Generate bool in_sample_Y2
# base_ms1ms2_pool['in_sample_Y2'] = (base_ms1ms2_pool['_merge'] == 'both')

# # WHY DO THIS – RIGHT_ONLY IS REMOVED SO NO DATA???
# # Generate bool newin13
# base_ms1ms2_pool['newin13'] = (base_ms1ms2_pool['_merge'] == 'right_only')

# # Generate bool attrit13
# base_ms1ms2_pool['attrit13'] = (base_ms1ms2_pool['_merge'] == 'left_only')

# base_ms1ms2_pool.drop(columns=['_merge'], inplace=True)


### Create Table 1.

In [7]:
df_tab1 = base_ms1ms2_pool.copy()
df_tab1['schoolfees_base'] = df_tab1['schoolfees_base']*1000

# var list for table 1
vars_list = [
    "male_base", "num_adults_base", "num_schoolchildren_base", "finished_primary_base",
    "finished_secondary_base", "cropland_base", "num_rooms_base", "schoolfees_base",
    "totcons_base", "logpercapcons_base", "total_cash_savings_base",
    "total_cash_savings_trimmed_base", "has_savings_acct_base", "taken_bank_loan_base",
    "taken_informal_loan_base", "liquidWealth_base", "wagepay_base",
    "businessprofitmonth_base", "price_avg_diff_pct_base",
    "price_expect_diff_pct_base", "harvest2011_base", "netrevenue2011_base",
    "netseller2011_base", "autarkic2011_base", "maizelostpct2011_base",
    "harvest2012_base", "correct_interest_base", "digit_recall_base",
    "maizegiver_base"
]


# AGAIN THE FOLLOWING IS DONE IN STATA BUT IS NOT NECESSARY (AT ALL!) SHOULD NOT BE NEEDED AS RIGHT_ONLY IS REMOVED
# Filter the DataFrame
# df_tab1 = base_ms1ms2_pool[base_ms1ms2_pool['newin13'] != True]

# Function to perform t-tests
def t_test_by_group(df, var, group_var='treat12'):
    group1 = df[df[group_var] == 0][var].dropna()
    group2 = df[df[group_var] == 1][var].dropna()
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
    return group1.mean(), group2.mean(), len(group1) + len(group2), t_stat, p_val

# Applying t-tests and collecting results
results = []
for var in vars_list:
    control_mean, treat_mean, obs, t_stat, p_val = t_test_by_group(df_tab1, var)
    std_diff = (treat_mean - control_mean) / np.sqrt(((len(df_tab1[df_tab1['treat12'] == 0][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 0][var], ddof=1) ** 2 + (len(df_tab1[df_tab1['treat12'] == 1][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 1][var], ddof=1) ** 2) / (len(df_tab1[df_tab1['treat12'] == 0][var]) + len(df_tab1[df_tab1['treat12'] == 1][var]) - 2))
    results.append([var, treat_mean, control_mean, obs, std_diff, p_val])

# Convert results to DataFrame for easier LaTeX conversion
results_df = pd.DataFrame(results, columns=['Variable', 'Treat Mean', 'Control Mean', 'Observations', 'Std Diff', 'P-value'])


latex_table1 = results_df.to_latex(index=False, float_format="%.3f")
print(latex_table1)



\begin{tabular}{lrrrrr}
\toprule
Variable & Treat Mean & Control Mean & Observations & Std Diff & P-value \\
\midrule
male_base & 0.296 & 0.334 & 1589 & -0.083 & 0.109 \\
num_adults_base & 3.004 & 3.196 & 1510 & -0.099 & 0.067 \\
num_schoolchildren_base & 2.998 & 3.072 & 1589 & -0.038 & 0.454 \\
finished_primary_base & 0.718 & 0.772 & 1490 & -0.122 & 0.019 \\
finished_secondary_base & 0.253 & 0.270 & 1490 & -0.039 & 0.460 \\
cropland_base & 2.441 & 2.398 & 1512 & 0.014 & 0.796 \\
num_rooms_base & 3.073 & 3.252 & 1511 & -0.072 & 0.219 \\
schoolfees_base & 27239.693 & 29813.631 & 1589 & -0.068 & 0.191 \\
totcons_base & 14970.862 & 15371.378 & 1437 & -0.032 & 0.550 \\
logpercapcons_base & 7.975 & 7.963 & 1434 & 0.019 & 0.721 \\
total_cash_savings_base & 5157.396 & 8021.499 & 1572 & -0.128 & 0.028 \\
total_cash_savings_trimmed_base & 4731.623 & 5389.836 & 1572 & -0.050 & 0.343 \\
has_savings_acct_base & 0.419 & 0.425 & 1589 & -0.012 & 0.815 \\
taken_bank_loan_base & 0.079 & 0.083 & 1589 & 

## Creating table 2, 3 and 4

In [8]:
ms1ms2_pooled_tab2 = ms1ms2_pooled.copy()

ms1ms2_pooled_tab2['strata_group'] = np.where(
    ms1ms2_pooled_tab2['MS'] == 2,
    ms1ms2_pooled_tab2['groupstrata'] + ms1ms2_pooled_tab2['strata_group'].max(),
    ms1ms2_pooled_tab2['strata_group']
)

ms1ms2_pooled_tab2['oafid'] = np.where(ms1ms2_pooled_tab2['MS'] == 2, 
                                       ms1ms2_pooled_tab2['fr_id'], 
                                       ms1ms2_pooled_tab2['oafid']
                                       )

ms1ms2_pooled_tab2 = ms1ms2_pooled_tab2.drop(columns=['purchaseval_trim', 'salesval_trim'])

for column in ['purchaseval', 'salesval', 'purchasequant', 'salesquant']:
    ms1ms2_pooled_tab2 = trim_quantiles(ms1ms2_pooled_tab2, column)
    

## Creating Table 7

In [9]:
# copy the raw data and create columns for treatment and interaction variable
ms1ms2_pooled_tab7 = ms1ms2_pooled.copy()
ms1ms2_pooled_tab7['z'] = pd.NA
ms1ms2_pooled_tab7['z_hi'] = pd.NA

### Running the first set of regressions

In [11]:
# list of treaments
treatments = ['treat12', 'treat13', 'treatMS1MS2']

# list of dependent variables
dependent_vars = ['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']

# list of changeing independent variables depending on the treatment
independent_vars = {
    'treat12': 'Y1round2 + Y1round3',
    'treat13': 'Y2round2 + Y2round3',
    'treatMS1MS2': 'Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3'
    }

# empty dictionary to store results
results = {}
pvals = {var: [] for var in ['z', 'hi', 'z_hi','z+z_hi']}

# Simulating the loop to replace variables and run regressions
for dv in dependent_vars:
    for treat in treatments:
        # Stata automatically omits the missing values in the regression – here we have to do it manually so we copy the data and drop variables
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc','interviewdate'])
        
        # setting treament variable
        df['z'] = df[treat] # setting z to the treatment variable
        
        # setting interaction variable
        df['z_hi'] = df[treat]*df['hi'] # setting z_hi to the interaction of the treatment hi saturation
        
        # setting the formula to run the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # Run the regression
        i = dependent_vars.index(dv)*len(treatments) + treatments.index(treat)
        results[f'model_{i}'] = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['subloc']})
        # print(results[f'model_{i}'].summary())

        # test the hypothesis that z + z_hi = 0
        hypothesis = 'z + z_hi = 0'
        t_test = results[f'model_{i}'].t_test(hypothesis)

        # store p-value round to 3 decimals
        pvals['z+z_hi'].append(np.round(t_test.pvalue,3))
        pvals['z'].append(np.round(results[f'model_{i}'].pvalues['z'],3))
        pvals['hi'].append(np.round(results[f'model_{i}'].pvalues['hi'],3))
        pvals['z_hi'].append(np.round(results[f'model_{i}'].pvalues['z_hi'],3))

### Running bootstrap regressions

In [47]:
# Simulating the loop to replace variables and run regressions
n_bootstraps = 1000

bootstrap_pvals = {key: {var: [] for var in ['z', 'hi', 'z_hi']} for key in results.keys()}

for dv in dependent_vars:
    for treat in treatments:
        # copying the raw data and dropping missing values
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc','interviewdate'])

        # setting the formula for the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # update the model key
        i = dependent_vars.index(dv)*len(treatments) + treatments.index(treat)
        model_key = f'model_{i}'

        df['z'] = df[treat] # setting z to the treatment variable
        df['z_hi'] = df[treat]*df['hi'] # setting z_hi to the interaction of the treatment hi saturation

        for j in range(n_bootstraps):
            # resample the data
            sample = resample(df, n_samples=len(df))

            # Run the regression
            model = smf.ols(formula, data=sample).fit(cov_type='cluster', cov_kwds={'groups': sample['subloc']})

            # add to array
            bootstrap_pvals[model_key]['z'].append(model.pvalues['z'])
            bootstrap_pvals[model_key]['hi'].append(model.pvalues['hi'])
            bootstrap_pvals[model_key]['z_hi'].append(model.pvalues['z_hi'])

In [42]:
bootstrap = {'z': [], 'hi': [], 'z_hi': []}

# Combine the averaging and new structure population in one loop
for model_data in bootstrap_pvals.values():
    for var in ['z', 'hi', 'z_hi']:
        mean_pval = np.mean(model_data[var])
        model_data[var] = mean_pval  # This line modifies the original dictionary, if preserving it is needed, handle appropriately
        bootstrap[var].append(mean_pval)

### Output code to LaTeX

In [45]:
# use stargazer to create a table
result_list = list(results.values())
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.title("Inventory, net revenues, and HH consumption effects")
stargazer.custom_columns(['Inventory', 'Net Revenues', 'Consumption'], [3, 3, 3])
stargazer.rename_covariates({'z': 'Treat', 'hi': 'High', 'z_hi': 'Treat*High'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'hi', 'z_hi'])
# add p-values as a rows 
stargazer.add_line('P-value T + TH = 0', pvals['z+z_hi'])
stargazer.add_line('P-value Treat', pvals['z'])
stargazer.add_line('P-value Treat Bootstrap', bootstrap['z'])
stargazer.add_line('P-value High', pvals['hi'])
stargazer.add_line('P-value High Bootstrap', bootstrap['hi'])
stargazer.add_line('P-value Treat*High', pvals['z_hi'])
stargazer.add_line('P-value Treat*High Bootstrap', bootstrap['z_hi'])


latex_table7 = stargazer.render_latex()

# edit the latex table to add row for telling if Y1 Y2 or Pooled after \\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\
latex_table7 = latex_table7.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \n \\\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\")
latex_table7 = latex_table7.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table7 = latex_table7.replace("Residual Std. Error", "% Residual Std. Error")
latex_table7 = latex_table7.replace("F Statistic", "% F Statistic")

print(latex_table7)


\begin{table}[!htbp] \centering
  \caption{Inventory, net revenues, and HH consumption effects}
\begin{tabular}{@{\extracolsep{5pt}}lccccccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{3}{c}{Inventory} & \multicolumn{3}{c}{Net Revenues} & \multicolumn{3}{c}{Consumption}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) 
 \\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\
\hline \\[-1.8ex]
 Treat & 0.759$^{***}$ & 0.546$^{***}$ & 0.740$^{***}$ & 1059.602$^{**}$ & 1193.768$^{*}$ & 1101.389$^{**}$ & 0.012$^{}$ & -0.051$^{}$ & -0.011$^{}$ \\
& (0.189) & (0.185) & (0.155) & (437.732) & (685.048) & (430.091) & (0.040) & (0.040) & (0.023) \\
 High & 0.124$^{}$ & -0.028$^{}$ & 0.017$^{}$ & 533.903$^{}$ & -152.603$^{}$ & 164.936$^{}$ & -0.003$^{}$ & -0.084$^{}$ & -0.047$^{}$ \\
& (0.355) & (0.219) & (0.241) & (551.179) & (558.948) & (479.685) & (0.051) & (0.053) & (0.043) \\
 Treat*High & -0.333$^{}$ & -0.065$^{}$ & -0.291$^{}$ & -1114.628$^{**